# Fase #2 Extracción con Selenium y BeautifulSoup

In [12]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep

from selenium.webdriver.common.by import By  
# By se utiliza para localizar elementos en una página web mediante diferentes métodos como XPATH, CSS_SELECTOR, etc.

import re

In [13]:
# Lista para almacenar las listas de IDs de película por año
listas_por_año_ids = {}

# Nombres de los archivos CSV por año, esto debe de estar en el escritorio para poder usarlos a menos que los tenga en el Visual
archivos_csv = ['../CSV/datos_2015.csv', 
                '../CSV/datos_2016.csv', 
                '../CSV/datos_2017.csv', 
                '../CSV/datos_2018.csv', 
                '../CSV/datos_2019.csv', 
                '../CSV/datos_2020.csv'] 

# Recorro el diccionario para ir a ver sus datos
for archivo in archivos_csv:
    año_ids = re.search(r'\d{4}', archivo).group()
    
    if año_ids:
        # Leer el archivo CSV
        df = pd.read_csv(archivo)
        
        # Filtrar las filas que tienen valores en la columna 'Id_pelicula'
        df_con_id = df.dropna(subset=['Id_pelicula'])
        
        # Crear una lista de los ID de película
        id_peliculas = df_con_id['Id_pelicula'].tolist()
        
        # Agregar la lista de ID de película a la lista principal
        listas_por_año_ids[año_ids] = id_peliculas
    else:
        print(f"No se pudo encontrar el año en el archivo: {archivo}")

# Imprimir la lista de listas
print(listas_por_año_ids.keys())


dict_keys(['2015', '2016', '2017', '2018', '2019', '2020'])


In [25]:
# función para extraer los detalles de las peliculas
def obtener_detalles_peliculas(tabla_movies):
    driver = webdriver.Chrome()

    # Abre la página web especificada
    driver.get("https://www.imdb.com/")
    driver.maximize_window()
    sleep(2)

    # Aceptamos las cookies
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    sleep(2)
    
    # Utilizo los CSV por año con el código que esta arriba, estan en el escritorio, si estuviera el el VS sería solo usarlo tal cual
    lista_peliculas = listas_por_año_ids['2020'][300:450] # le indico las posiciones o hasta donde quiero que llegue
    lista_detalles_peliculas = [] 
    
    
    for id in lista_peliculas:
            
        try:
            driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER) #Barra de busqueda
            sleep(2)
                                           
            try:
                try:
                    nombre_pelicula = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
                except:
                    nombre_pelicula = "No info"
            except:
                pass
            

            try:
                try: 
                    director = driver.find_element('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div').text
                except:
                    director = "No info"         
            except:
                pass  

            
            try: 
                puntuacion_1 = driver.find_element('xpath', '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
                puntuacion_1 = puntuacion_1.replace(",",".")
                puntuacion = float(puntuacion_1)              
                
            except:
                puntuacion = "No info"

                 
            try:
                guionista1 = driver.find_element("xpath", '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul').text
                guionistas = re.sub(r'(?<=[a-z])(?=[A-Z])', ', ', guionista1)
            except:
                try:
                    driver.find_element("xpath", '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/div[2]/div/ul/li[2]/div/ul')
                except:
                    guionistas = "No info"

    
            try:
                argumento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
            except:
                try:
                    argumento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/p/span[3]').text
                    if argumento == '':
                        argumento = None
                except:
                    argumento = "No info"
                
            
            try:
                # Buscar la duración solo una vez
                duracion_encontrada = False
                for i in range(2,4): # apunta el rango de la información de loas horas y minutos que hay en la pag
                    duracion_element = driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[{i}]')
                    duracion = duracion_element.text
                    # la h, min, minutes y m es lo que aparece en la pag para sacar sus datos
                    if 'h' in duracion or 'min' in duracion or 'minutes' in duracion or 'm' in duracion:
                        duracion_encontrada = True
                        break  # Salir del bucle si se encuentra una duración válida
                
                # Si no se encuentra ninguna duración válida, establecerla como "No info"
                if not duracion_encontrada:
                    duracion = "No info"
            except:
                #duracion = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)').text
                duracion = "No info"

            sleep(2)
            # Guardo en tupla
            tupla_detalles = (nombre_pelicula, director, puntuacion, guionistas, argumento, duracion)
            lista_detalles_peliculas.append(tupla_detalles)           
    
        except:
            pass
        
        driver.back()
        
    #driver.quit()

    return lista_detalles_peliculas


# 2020

In [26]:
datos_2020_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [27]:
datos_2020_caracteristicas = pd.DataFrame(datos_2020_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [28]:
datos_2020_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,The Big 3,Tye Banks,9.2,"Bo Alambis, Ampston Hews, Brian Isley",Liquor House Comedy presents the Big 3 featuri...,46m
1,Moonboy,Christopher Logan,No info,Christopher Logan,Moonboy is a catalyst in other people's lives ...,No info
2,My Cousin,Jan Kounen,5.3,"Fabrice Roger-Lacan, Jan Kounen, Vincent Lindon",The CEO of a big international wine company mu...,1h 44m
3,Whenever You're Ready,Jenna Harder,No info,Jenna Harder,A darkly comedic look at a day in the life of ...,No info
4,1 Euro/9 Minutos,Sara Muñoz BautistaRebeca Lasheras,No info,"Sara Muñoz Bautista, Rebeca Lasheras",,12m
...,...,...,...,...,...,...
145,F.M. Live,Stephen Province,No info,Jessica Stamper,,No info
146,The Dummy Factor,O. Corbin Saleken,4.8,O. Corbin Saleken,A 12-year-old boy investigates the disappearan...,1h 21m
147,How I Became a Super Hero,Douglas Attal,5.8,"Douglas Attal, Melisa Godet, Charlotte Sanson",While superheroes have assimilated into Parisi...,1h 37m
148,Boszorkányház,István Nyíri Kovács,6.6,"Gábor Kertai, István Nyíri Kovács",,No info


In [29]:
datos_2020_caracteristicas.to_csv('Caracteristicas_2020.csv', index=False)

# 2019

In [21]:
datos_2019_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [22]:
datos_2019_caracteristicas = pd.DataFrame(datos_2019_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [23]:
datos_2019_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,La fête espagnole,Germaine Dulac,7.0,Louis Delluc,"Coveted by two different men, a woman turns to...",1h 7m
1,Sha Ou,Nuanxing Zhang,6.6,"Tuo Li, Nuanxing Zhang",Sha Ou is a volleyball player and works hard a...,1h 24m
2,Mariette in Ecstasy,John Bailey,7.3,Ron Hansen,Set in a turn-of-the-century religious communi...,1h 41m
3,Husband and Wife,No info,5.9,Ron Hansen,Li Ke and Zhang Ying marry and overcome diffic...,No info
4,Motherless Brooklyn,Edward Norton,6.8,"Edward Norton, Jonathan Lethem","In 1950s New York, Lionel, a private detective...",2h 24m
...,...,...,...,...,...,...
95,A Christmas Cracker,Maximilian Mena,No info,Maximilian Mena,"During the height of festive consumerism, Holl...",15m
96,It Was the Worst,No info,No info,Maximilian Mena,None,No info
97,Chicago I'm Home,Barry Brewer,7.4,Barry Brewer,Taped in front of two sold out crowds at the S...,1h 18m
98,Out of Touch,No info,No info,Barry Brewer,None,No info


In [24]:
datos_2019_caracteristicas.to_csv('Caracteristicas_2019.csv', index=False)

# 2018

In [15]:
datos_2018_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [16]:
datos_2018_caracteristicas = pd.DataFrame(datos_2018_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [17]:
datos_2018_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,The Other Side of the Wind,Orson Welles,6.7,"Orson Welles, Oja Kodar",At a media-swamped party to celebrate his 70th...,2h 2m
1,Reverse Heaven,Stuart Paul,3.0,Stuart Paul,"In 1990s Los Angeles, danger lurks around the ...",1h 44m
2,To Chase a Million,Pat Jackson,6.6,"Richard Bradford, Ron Randell, Anton Rodgers",A lone shark bounty hunter pits himself agains...,1h 37m
3,Aamaar Bhuvan,Mrinal Sen,7.8,"Afsar Ahmed, Mrinal Sen, H. Banerjee",A triangle love story between viz Meher (Kaush...,1h 42m
4,Fahrenheit 451,Ramin Bahrani,5.0,"Ray Bradbury, Ramin Bahrani, Amir Naderi","In a terrifying care-free future, a young man,...",1h 40m
...,...,...,...,...,...,...
95,Air Asia Commercial,No info,No info,Ivan Martins,None,No info
96,Eric Offline,No info,No info,Ivan Martins,"Eric's social media image couldn't be better, ...",20m
97,Verano No Miente,Ernesto Santisteban,5.4,Ernesto Santisteban,In a countryside house located in the Ecuadore...,2h
98,Freej Al Taibeen,Ahmed Zain,5.3,Ahmed Zain,Local rivals gear up to compete in a tradition...,1h 40m


In [19]:
datos_2018_caracteristicas.to_csv('Caracteristicas_2018.csv', index=False)

# 2017

In [8]:
datos_2017_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [9]:
datos_2017_caracteristicas = pd.DataFrame(datos_2017_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [10]:
datos_2017_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,Lunacy,Carl Abrahamsson,No info,Vanessa Sinclair,Praising lunar forces and witches from all tim...,44m
1,It's Lonely Down Here,Kreigh Carter,No info,Kreigh Carter,A Young girl in a broken home finds kinship in...,No info
2,Hannah,Anh Be,No info,Zimf,The story is about a young man named Le Minh V...,7m
3,The Girl From Future,No info,No info,No info,It is a film directed by Wu Qin.The film is ab...,1h 38m
4,Leveler,Eric Warwaruk,No info,Eric Warwaruk,A young genius teen girl abused emotionally an...,12m
...,...,...,...,...,...,...
95,Cultos,Martha Poly Vil,No info,Angel Inzunza,A brief presentation of Sindicato del Terror; ...,3m
96,Innocent Souls,Alejandro López,3.8,Alejandro López,,2m
97,Dearest Dolly,Matthew Mark Hunter,No info,Matthew Mark Hunter,A young man walks into the wrong place at the ...,No info
98,Forbidden Woods,Matthew Mark Hunter,No info,Matthew Mark Hunter,A young man searches for a cult that may be in...,No info


In [11]:
datos_2017_caracteristicas.to_csv('Caracteristicas_2017.csv', index=False)

# 2016

In [35]:
datos_2016_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [36]:
datos_2016_caracteristicas = pd.DataFrame(datos_2016_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [37]:
datos_2016_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,Jase Bond - La mort ne meurt jamais,Maxime Scharwath,No info,"Damien Moriggi, Olivia Borgeaud, Maximilien Ze...",,No info
1,Mercenarios en acción,Pablo Martínez,No info,"Torres Héctor, Martínez Pablo",,45m
2,Doodh Ka Karz,No info,No info,"Torres Héctor, Martínez Pablo",None,No info
3,The Secret,Carlos Burley,No info,Carlos Burley,"Aiza is a hit-woman, hired by the worlds most ...",15m
4,Il vizio,Evis Nako,No info,Evis Nako,,24m
...,...,...,...,...,...,...
95,Serial,No info,No info,"Lorenzo Paviano, Raffaele Picchio, Alessandro ...",None,No info
96,Derelict,Leonard Nicholas Ruiz,No info,Leonard Nicholas Ruiz,,No info
97,16 Fight Against Ragging,Lagdhirsinh Jadeja,No info,Jaydeepsinh V Zala,The story focuses on the issue of ragging carr...,2h 20m
98,The Noise,No info,No info,Jaydeepsinh V Zala,None,No info


In [38]:
datos_2016_caracteristicas.to_csv('Caracteristicas_2016.csv', index=False)

# 2015

In [27]:
datos_2015_caracteristicas = obtener_detalles_peliculas(listas_por_año_ids)

In [28]:
datos_2015_caracteristicas = pd.DataFrame(datos_2015_caracteristicas, columns=['Nombre_Pelicula', 'Director', 'Puntuación', 'Guionistas', 'Argumento', 'Duración'])

In [29]:
datos_2015_caracteristicas

,Nombre_Pelicula,Director,Puntuación,Guionistas,Argumento,Duración
0,Shulie,Elisabeth Subrin,6.4,Elisabeth Subrin,A shot-by-shot reenactment of an unreleased 19...,37m
1,Snowblind,No info,7.0,No info,"Revealing the entrepreneurial ingenuity, paran...",No info
2,Chatô: O Rei do Brasil,Guilherme Fontes,5.9,"João Emanuel Carneiro, Guilherme Fontes, Matth...","The story of Assis Chateaubriand, the first ma...",1h 42m
3,Transeúntes,Luis Aller,6.7,"Luis Aller, María José Garcia, María José García","Passers-by is a constructivist, multiple, frag...",1h 41m
4,The Stanford Prison Experiment,Kyle Patrick Alvarez,6.8,"Tim Talbott, Philip Zimbardo","In 1971, twenty-four male students are selecte...",2h 2m
...,...,...,...,...,...,...
95,Palasik,Dedy Mercy,5.5,Baskoro Adi Wuryanto,A pregnant woman who was accused by residents ...,1h 18m
96,Kernel in My Throat,Domenic Migliore,No info,"Patch the Beagle, Domenic Migliore, Alex Quatt...",,3m
97,Penghuni Lain,Roman Malik,No info,"Delbin Clyte, Helfi C.H. Kardit",A married couple and their daughter move to a ...,1h 23m
98,Tes Nyali,No info,No info,"Delbin Clyte, Helfi C.H. Kardit",None,No info


In [30]:
datos_2015_caracteristicas.to_csv('Caracteristicas_2015.csv', index=False)